In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
pd.options.mode.chained_assignment = None  # default='warn'

In [104]:
# Other imports

# Get Data


In [105]:
utilData = pd.read_csv('./Util_Run1.csv')
utilData['memLoad'] = utilData['memUse']*100/utilData['memTot']
utilData['memScore'] = utilData['memLoad']*utilData['memfreq']/825000000
utilData['memF'] = utilData['memfreq']*100/825000000
utilData['cpuScore'] = utilData['cpuload']*utilData['cpufreq']/1400000
utilData['cpuF'] = utilData['cpufreq']*100/1000000

In [106]:
# Some data modification
utilData

,memUse,memTot,memfreq,cpuid,cpuload,cpufreq,memLoad,memScore,memF,cpuScore,cpuF
0,604872,4612865,165000000,0,31,600000,13.112718,2.622544,20.000000,13.285714,60.0
1,675,28712,275000000,0,4,200000,2.350933,0.783644,33.333333,0.571429,20.0
2,1191888,12624777,413000000,0,8,200000,9.440864,4.726154,50.060606,1.142857,20.0
3,773492,8196989,275000000,0,9,300000,9.436294,3.145431,33.333333,1.928571,30.0
4,1144,57890,206000000,0,7,300000,1.976162,0.493442,24.969697,1.500000,30.0
...,...,...,...,...,...,...,...,...,...,...,...
2669,807320,11546069,413000000,0,4,200000,6.992163,3.500319,50.060606,0.571429,20.0
2670,195717,2817255,275000000,0,7,300000,6.947081,2.315694,33.333333,1.500000,30.0
2671,1344,50569,275000000,0,4,300000,2.657755,0.885918,33.333333,0.857143,30.0
2672,1344,50569,275000000,0,5,300000,2.657755,0.885918,33.333333,1.071429,30.0


In [107]:
# utilData.shift(1)
utilData.shape[0]

2674

# Fuzzy AR-MA


In [115]:
# def ART(p,endog,exog=None,data_weight=None):
    
#     df = pd.DataFrame()
#     df['endog'] = endog
#     if exog!= None:
#         df['exog'] = exog
    
    
df = pd.DataFrame()
# df['Weight'] = np.ones(utilData.shape[0])
df['Value'] = utilData['cpuScore'].diff(1)
df['Weight'] = np.linspace(0,1,utilData.shape[0])
df['exog'] = utilData['memLoad'].shift(1)
# df.iloc[:,1]
df

,Value,Weight,exog
0,NaN,0.000000,NaN
1,-12.714286,0.000374,13.112718
2,0.571429,0.000748,2.350933
3,0.785714,0.001122,9.440864
4,-0.428571,0.001496,9.436294
...,...,...,...
2669,-0.714286,0.998504,6.992163
2670,0.928571,0.998878,6.992163
2671,-0.642857,0.999252,6.947081
2672,0.214286,0.999626,2.657755


In [116]:
from statsmodels.tools.eval_measures import rmspe
def FuzzyARX(p,df):
    df_temp = pd.DataFrame()
    df_temp['Value'] = df['Value']

    #Generating the lagged p terms
    for i in range(1,p+1):
        df_temp['Shifted_values_%d' % i ] = df_temp['Value'].shift(i)    
    df_temp['exog'] = df['exog']
    df_temp['Weight'] = df['Weight']
    train_size = (int)(1.0 * df_temp.shape[0])
    #Breaking data set into test and training
    df_train = pd.DataFrame(df_temp[0:train_size])
#     df_test = pd.DataFrame(df_temp[train_size:df.shape[0]])
    df_train_2 = df_train.dropna()
    X_train = df_train_2.iloc[:,1:p+2].values.reshape(-1,p+1)
    #Y contains the value,it is the first column
    y_train = df_train_2.iloc[:,0].values.reshape(-1,1)
    print(y_train.shape[0])
    sample_weight = df_train_2['Weight']
    #Running linear regression to generate the coefficents of lagged terms
    from sklearn.linear_model import LinearRegression
    lr = LinearRegression().fit(X_train,y_train,sample_weight)
#     lr.fit(X_train,y_train,sample_weight)
    theta  = lr.coef_.T
    intercept = lr.intercept_
#     df_train_2['Predicted_Values'] = X_train.dot(lr.coef_.T) + lr.intercept_
#     df_train_2['Predicted_Values'] = lr.predict(X_train)
    # df_train_2[['Value','Predicted_Values']].plot()
    print('R-means',lr.score(X_train,y_train),'theta',lr.coef_.T,'const',lr.intercept_)
#     X_test = df_test.iloc[:,1:p+2].values.reshape(-1,p+1)
#     df_test['Predicted_Values'] = X_test.dot(lr.coef_.T) + lr.intercept_
#     df_test['Predicted_Values'] = lr.predict(X_test)
    # df_test[['Value','Predicted_Values']].plot()
#     RMSE = np.sqrt(mean_squared_error(df_test['Value'], df_test['Predicted_Values']))
#     RMSE = rmspe(df_test['Value'],df_test['Predicted_Values'])

#     print("The RMSE is :", RMSE,", Value of p : ",p)
#     return [df_train_2,df_test,theta,intercept,RMSE]


    
    print(X_train)
    import statsmodels.api as sm
    X_sm = sm.add_constant(X_train)
    print(X_sm)
#     model = sm.WLS(y_train, X_sm, weights = sample_weight)
#     results = model.fit()
#     print(results.summary())
    return [X_sm,y_train,sample_weight]

In [117]:
res = FuzzyARX(3,df)

2670
R-means 0.06717140865048932 theta [[-0.26647062]
 [-0.07495777]
 [-0.06671497]
 [ 0.04464993]] const [-0.61430784]
[[  0.78571429   0.57142857 -12.71428571   9.43629423]
 [ -0.42857143   0.78571429   0.57142857   1.97616169]
 [ -1.07142857  -0.42857143   0.78571429   6.18651917]
 ...
 [  0.92857143  -0.71428571   0.42857143   6.94708147]
 [ -0.64285714   0.92857143  -0.71428571   2.65775475]
 [  0.21428571  -0.64285714   0.92857143   2.65775475]]
[[  1.           0.78571429   0.57142857 -12.71428571   9.43629423]
 [  1.          -0.42857143   0.78571429   0.57142857   1.97616169]
 [  1.          -1.07142857  -0.42857143   0.78571429   6.18651917]
 ...
 [  1.           0.92857143  -0.71428571   0.42857143   6.94708147]
 [  1.          -0.64285714   0.92857143  -0.71428571   2.65775475]
 [  1.           0.21428571  -0.64285714   0.92857143   2.65775475]]


In [118]:
X,y,w = res[0],res[1],res[2]
X[0]

array([  1.        ,   0.78571429,   0.57142857, -12.71428571,
         9.43629423])

In [119]:
import statsmodels.api as sm
model = sm.WLS(y, X,weights = w)
results = model.fit()
X_t = [[0.6,  -1.0, 0.9, 7.1],[0.8,  -0.0, 1.9, 7.4]]
X_t = np.array(X_t).reshape(-1,4)
X_t

array([[ 0.6, -1. ,  0.9,  7.1],
       [ 0.8, -0. ,  1.9,  7.4]])

In [120]:
X_t = sm.add_constant(X_t)
X_t

array([[ 1. ,  0.6, -1. ,  0.9,  7.1],
       [ 1. ,  0.8, -0. ,  1.9,  7.4]])

In [121]:
results.get_prediction(X_t).predicted_mean

array([-0.44226145, -0.62383333])

In [128]:
results.get_prediction(X_t).summary_frame()

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,-0.442261,0.222355,-0.878267,-0.006256,-15.534188,14.649665
1,-0.623833,0.223999,-1.063063,-0.184604,-15.715854,14.468187


In [122]:
results.pvalues

array([1.67349567e-02, 5.35601581e-42, 1.64544534e-04, 5.35945678e-04,
       6.15651442e-05])

In [123]:
results.params

array([-0.61430784, -0.26647062, -0.07495777, -0.06671497,  0.04464993])

In [124]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.073
Model:                            WLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     52.46
Date:                Thu, 13 Jul 2023   Prob (F-statistic):           1.34e-42
Time:                        17:14:59   Log-Likelihood:                -10557.
No. Observations:                2670   AIC:                         2.112e+04
Df Residuals:                    2665   BIC:                         2.115e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6143      0.257     -2.394      0.017      -1.117      -0.111
x1            -0.2665      0.019    -13.818      0.000      -0.304      -0.229
x2            -0.0750      0.020     -3.773      0.000      -0.114      -0.036
x3            -0.0667      0.019     -3.466      0.001      -0.104      -0.029
x4             0.0446      0.011      4.013      0.000       0.023       0.066
==============================================================================
Omnibus:                      977.738   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           160729.120
Skew:                          -0.604   Prob(JB):                         0.00
Kurtosis:                      40.991   Cond. No.                         28.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [125]:
val = FuzzyARX(4,df)

2669
R-means 0.06655541646245577 theta [[-0.26809823]
 [-0.07675103]
 [-0.07323876]
 [-0.02478194]
 [ 0.04409747]] const [-0.60764485]
[[ -0.42857143   0.78571429   0.57142857 -12.71428571   1.97616169]
 [ -1.07142857  -0.42857143   0.78571429   0.57142857   6.18651917]
 [  0.64285714  -1.07142857  -0.42857143   0.78571429  10.52235157]
 ...
 [  0.92857143  -0.71428571   0.42857143  -0.64285714   6.94708147]
 [ -0.64285714   0.92857143  -0.71428571   0.42857143   2.65775475]
 [  0.21428571  -0.64285714   0.92857143  -0.71428571   2.65775475]]
[[  1.          -0.42857143   0.78571429   0.57142857 -12.71428571
    1.97616169]
 [  1.          -1.07142857  -0.42857143   0.78571429   0.57142857
    6.18651917]
 [  1.           0.64285714  -1.07142857  -0.42857143   0.78571429
   10.52235157]
 ...
 [  1.           0.92857143  -0.71428571   0.42857143  -0.64285714
    6.94708147]
 [  1.          -0.64285714   0.92857143  -0.71428571   0.42857143
    2.65775475]
 [  1.           0.21428571  -0

In [ ]:
val = FuzzyARX(4,df)

In [ ]:
val = FuzzyARX(4,df)